<a href="https://colab.research.google.com/github/eschwarzbeckf/taxi/blob/main/taxi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/eschwarzbeckf/taxi.git
!cd taxi
!pip install -r requirements.txt

fatal: destination path 'taxi' already exists and is not an empty directory.
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [2]:
import gymnasium as gym

In [5]:
env = gym.make('Taxi-v3')

np.int64(500)

fatal: destination path 'taxi' already exists and is not an empty directory.
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
